In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 100% !important; }</style>"))

## Forecasting Stock Prices of Volkswagen

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import yfinance

In [3]:
raw_data = yfinance.download(tickers= "VOW3.DE", interval= "1d",
                             group_by='ticker', auto_adjust=True, treads=True)
df= raw_data.copy()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
1998-07-22,33.009003,34.005072,33.009003,33.766056,56336
1998-07-23,34.138089,34.403602,32.278446,32.278446,95571
1998-07-24,31.879916,32.543962,31.295368,32.145432,154423
1998-07-27,32.145429,32.411463,31.082852,31.215868,62875
1998-07-28,30.950353,32.118928,30.232789,30.471804,116194
...,...,...,...,...,...
2021-08-25,199.380005,202.250000,198.000000,199.699997,559333
2021-08-26,198.600006,201.449997,197.520004,200.000000,621505
2021-08-27,200.050003,200.800003,198.699997,200.750000,458406


## Defining Key Dates

In [5]:
## Starting Dates
start_date = "2009-04-05"

# First official Announcement - 49.9 %
ann_1 = "2009-12-09"

# Second Official Announcement - 50.1 %
ann_2 = '2012-07-05'

#Ending Date
end_date = "2014-01-01"

#Dieselgate Scandle
d_gate = "2015-09-20"

## Data Preporcessing

In [6]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [7]:
# Creating returns
df['Return'] = df.Close.pct_change(1).mul(100)

# Creating Squared Returns 
df['Return_sq'] = df.Return.mul(df.Return)

#Assigning the frequencies and filling na values 
df = df.asfreq('b')
df = df.fillna(method='ffill')

#Removing Surplus Data
#l df['VOW3.DE'], df['PAH3.DE'], df['BMW.DE']

In [10]:
df = df[['Close', 'Volume', 'Return', 'Return_sq']]
df

,Close,Volume,Return,Return_sq
Date,,,,
1998-07-22,33.766056,56336.0,NaN,NaN
1998-07-23,32.278446,95571.0,-4.405637,19.409637
1998-07-24,32.145432,154423.0,-0.412085,0.169814
1998-07-27,31.215868,62875.0,-2.891744,8.362182
1998-07-28,30.471804,116194.0,-2.383609,5.681593
...,...,...,...,...
2021-08-25,199.699997,559333.0,0.472933,0.223666
2021-08-26,200.000000,621505.0,0.150227,0.022568
2021-08-27,200.750000,458406.0,0.375000,0.140625


In [ ]:
df.Close.plot(figsize=)